# Fairness Through Unawareness - Recruiting data

This notebook contains the implementation of the common pre-processing intervention called Fairness Through Unawareness (FTU) in which the protected attribute is not included as a feature in the training data. Besides being considered as an intervention, FTU can also be considered as a fairness notion, which is consistent with disparate treatment.

Although FTU is often applied by industry practitioners, its effect in terms of reducing unfairness is limited since information on protected attributed can still be contained elsewhere in the data. More precisely, there may be features which are highly correlated with the protected attributes and therefore act as proxies for them.

We consider the effect of applying FTU for a number of observational group fairness notions.

In [ ]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
from fairlearn.metrics import (
    demographic_parity_difference,
    equalized_odds_difference,
)
from sklearn.neural_network import MLPClassifier  # noqa
from helpers.metrics import accuracy
from helpers.plot import group_bar_plots, group_box_plots, calibration_curves

## Load data

We have committed preprocessed data to the repository for reproducibility and we load it here. Check out hte preprocessing notebook for details on how this data was obtained.

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "recruiting"

train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

## Load original model

For maximum reproducibility we can also load the baseline model from disk, but the code used to train can be found in the baseline model notebook.

In [ ]:
baseline_model = joblib.load(
    artifacts_dir / "models" / "recruiting" / "baseline.pkl"
)

Get predictions on the test data

In [ ]:
bl_test_probs = baseline_model.predict_proba(
    test.drop("employed_yes", axis=1)
)[:, 1]
bl_test_labels = (bl_test_probs > 0.5).astype(float)

## Learn model under FTU

Generate FTU data sets

In [ ]:
train_ftu = train.drop("race_white", axis=1).copy()
val_ftu = val.drop("race_white", axis=1).copy()
test_ftu = test.drop("race_white", axis=1).copy()

Learn model on FTU training data

In [ ]:
# ftu_model = MLPClassifier(hidden_layer_sizes=(100, 100), early_stopping=True)
# ftu_model.fit(train_ftu.drop(columns="employed_yes"), train.employed_yes)

In [ ]:
ftu_model = joblib.load(artifacts_dir / "models" / "recruiting" / "ftu.pkl")

Generate prediction via learnt FTU model on test data

In [ ]:
test_probs = ftu_model.predict_proba(test_ftu.drop("employed_yes", axis=1))[
    :, 1
]
test_pred_labels = (test_probs > 0.5).astype(float)

## Demographic parity

We first address the effect on demographic parity using FTU.

In [ ]:
test_race_white = test.race_white.values
test_employed_yes = test.employed_yes.values
mask = test_race_white == 1

# baseline metrics
bl_test_acc = accuracy(test_employed_yes, bl_test_probs)
bl_test_dpd = demographic_parity_difference(
    test.employed_yes, bl_test_labels, sensitive_features=test_race_white,
)

# new model metrics
test_acc = accuracy(test_employed_yes, test_probs)
test_dpd = demographic_parity_difference(
    test.employed_yes, test_pred_labels, sensitive_features=test_race_white,
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline demographic parity: {bl_test_dpd:.3f}")
print(f"Demographic parity: {test_dpd:.3f}\n")

In [ ]:
dp_box = group_box_plots(
    np.concatenate([bl_test_probs, test_probs]),
    np.tile(test.race_white.map({0: "Black", 1: "White"}), 2),
    groups=np.concatenate(
        [np.zeros_like(bl_test_probs), np.ones_like(test_probs)]
    ),
    group_names=["Baseline", "Kamishima"],
    title="Distribution of scores by race",
    xlabel="Scores",
    ylabel="Method",
)
dp_box

## Equalised odds

Let us now evaluate equalised odds for the FTU model on the test data.

In [ ]:
test_race = test.race_white.values
test_employed = test.employed_yes.values
mask = test_race == 1

# baseline metrics
bl_test_acc = accuracy(test_employed, bl_test_probs)
bl_test_eod = equalized_odds_difference(
    test_employed, bl_test_labels, sensitive_features=test_race,
)

# new model metrics
test_acc = accuracy(test_employed, test_pred_labels)
test_eod = equalized_odds_difference(
    test_employed, test_pred_labels, sensitive_features=test_race,
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline equalised odds (dist.): {bl_test_eod:.3f}")
print(f"Equalised odds (dist.): {test_eod:.3f}\n")

In [ ]:
bl_eo_bar = group_bar_plots(
    bl_test_probs,
    test.race_white.map({0: "Black", 1: "White"}),
    groups=test.employed_yes,
    group_names=["Not employed", "Employed"],
    title="Predictions by race and outcome",
    xlabel="Proportion predicted successful",
    ylabel="Outcome",
)
bl_eo_bar

In [ ]:
eo_bar = group_bar_plots(
    test_pred_labels,
    test.race_white.map({0: "Black", 1: "White"}),
    groups=test.employed_yes,
    group_names=["Not employed", "Employed"],
    title="Predictions by race and outcome",
    xlabel="Proportion predicted successful",
    ylabel="Outcome",
)
eo_bar

## Equal opportunity

Let us now evaluate equal opportunity for the FTU model on the test data.

In [ ]:
test_race = test.race_white.values
test_employed = test.employed_yes.values
mask = test_race == 1

# baseline metrics
bl_test_acc = accuracy(test_employed, bl_test_probs)
bl_test_eoppd = equalized_odds_difference(
    test_employed[test.employed_yes == 1],
    bl_test_labels[test.employed_yes == 1],
    sensitive_features=test_race[test.employed_yes == 1],
)

# new model metrics
test_acc = accuracy(test_employed, test_pred_labels)
test_eoppd = equalized_odds_difference(
    test_employed[test.employed_yes == 1],
    test_pred_labels[test.employed_yes == 1],
    sensitive_features=test_race[test.employed_yes == 1],
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline equal opportunity: {bl_test_eoppd:.3f}")
print(f"Equal opportunity: {test_eoppd:.3f}\n")

In [ ]:
mask = test.employed_yes == 1

eopp_bar = group_bar_plots(
    np.concatenate([bl_test_labels[mask], test_pred_labels[mask]]),
    np.tile(test.race_white[mask].map({0: "Black", 1: "White"}), 2),
    groups=np.concatenate(
        [np.zeros_like(bl_test_probs[mask]), np.ones_like(test_probs[mask])]
    ),
    group_names=["Baseline", "FTU"],
    title="Mean prediction for being employed by race",
    xlabel="Proportion predicted being employed",
    ylabel="Method",
)
eopp_bar

## Calibration

Let us now evaluate calibration for the FTU model on the test data.

In [ ]:
calibration_curves(
    test.employed_yes,
    bl_test_probs,
    test.race_white.map({0: "Black", 1: "White"}),
    title="Baseline calibration by race",
    xlabel="Score",
    ylabel="Proportion positive outcome",
)

In [ ]:
calibration_curves(
    test.employed_yes,
    test_probs,
    test.race_white.map({0: "Black", 1: "White"}),
    title="FTU calibration by race",
    xlabel="Score",
    ylabel="Proportion positive outcome",
)